# Exploration des données

Ce notebook explore les différentes sources de données pour le projet d'analyse de l'impact des feux sur l'albédo des glaciers.

In [ ]:
# Imports de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Exploration des données RGI

In [ ]:
# Charger les glaciers RGI
rgi_path = Path('../data/rgi/RGI60-02.shp')

if rgi_path.exists():
    glaciers = gpd.read_file(rgi_path)
    print(f"Nombre total de glaciers: {len(glaciers)}")
    print(f"Colonnes disponibles: {list(glaciers.columns)}")
    
    # Filtrer pour l'ouest canadien
    canada_glaciers = glaciers[glaciers['CenLon'] < -115]
    print(f"\nGlaciers ouest canadien: {len(canada_glaciers)}")
    
    # Statistiques
    print("\nStatistiques des surfaces:")
    print(canada_glaciers['Area'].describe())
else:
    print("Fichier RGI non trouvé. Exécuter d'abord scripts/download_data.sh")

In [ ]:
# Visualiser la distribution spatiale
if 'canada_glaciers' in locals():
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Carte de base
    canada_glaciers.plot(column='Area', 
                        cmap='Blues',
                        legend=True,
                        ax=ax)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Distribution des glaciers RGI - Ouest Canadien')
    plt.tight_layout()

## 2. Exploration des données RAQDPS (exemple)

In [ ]:
# Exemple de fichier RAQDPS
raqdps_example = Path('../data/raqdps/20230715/2023071512_000.nc')

if raqdps_example.exists():
    ds = xr.open_dataset(raqdps_example)
    print("Variables disponibles:")
    print(list(ds.data_vars))
    
    print("\nDimensions:")
    print(ds.dims)
    
    # Afficher les métadonnées d'une variable
    if 'BC_dep' in ds:
        print("\nMétadonnées BC_dep:")
        print(ds['BC_dep'])
else:
    print("Fichier RAQDPS exemple non trouvé")

In [ ]:
# Visualiser le champ de dépôt BC
if 'ds' in locals() and 'BC_dep' in ds:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Sélectionner la région des glaciers
    bc_dep = ds['BC_dep'].sel(
        lat=slice(48, 60),
        lon=slice(-130, -110)
    )
    
    # Tracer
    im = bc_dep.plot(ax=ax, cmap='YlOrRd', 
                     cbar_kwargs={'label': 'Dépôt BC (kg/m²/s)'})
    
    ax.set_title('Exemple de champ de dépôt BC - RAQDPS')
    plt.tight_layout()

## 3. Analyse du glacier Athabasca

In [ ]:
# Localisation du glacier Athabasca
athabasca_info = {
    'name': 'Athabasca Glacier',
    'lat': 52.185,
    'lon': -117.252,
    'area_km2': 6.0
}

# Créer un buffer autour du glacier
from shapely.geometry import Point
athabasca_point = Point(athabasca_info['lon'], athabasca_info['lat'])
athabasca_buffer = athabasca_point.buffer(0.1)  # 0.1 degré

print(f"Glacier Athabasca:")
print(f"  Position: {athabasca_info['lat']}°N, {athabasca_info['lon']}°W")
print(f"  Surface: {athabasca_info['area_km2']} km²")

In [ ]:
# Extraire les valeurs RAQDPS pour Athabasca
if 'ds' in locals() and 'BC_dep' in ds:
    # Sélectionner la région
    lat_min, lat_max = 52.1, 52.3
    lon_min, lon_max = -117.4, -117.1
    
    bc_athabasca = ds['BC_dep'].sel(
        lat=slice(lat_min, lat_max),
        lon=slice(lon_min, lon_max)
    )
    
    # Statistiques
    print(f"Dépôt BC sur la région Athabasca:")
    print(f"  Moyenne: {float(bc_athabasca.mean()):.2e} kg/m²/s")
    print(f"  Maximum: {float(bc_athabasca.max()):.2e} kg/m²/s")
    print(f"  Minimum: {float(bc_athabasca.min()):.2e} kg/m²/s")

## 4. Exploration des données de feux

In [ ]:
# Charger les données de feux MODIS
fire_path = Path('../data/fires/MODIS_fires_Canada_2023.csv')

if fire_path.exists():
    fires = pd.read_csv(fire_path)
    print(f"Nombre total de détections: {len(fires)}")
    print(f"\nColonnes: {list(fires.columns)}")
    
    # Filtrer pour l'ouest canadien et l'été 2023
    fires['acq_date'] = pd.to_datetime(fires['acq_date'])
    
    summer_fires = fires[
        (fires['longitude'] < -110) & 
        (fires['latitude'] > 48) &
        (fires['acq_date'] >= '2023-06-01') &
        (fires['acq_date'] <= '2023-09-30')
    ]
    
    print(f"\nFeux ouest canadien (été 2023): {len(summer_fires)}")
else:
    print("Fichier de feux non trouvé")

In [ ]:
# Visualiser les feux et les glaciers ensemble
if 'canada_glaciers' in locals() and 'summer_fires' in locals():
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Glaciers
    canada_glaciers.plot(color='lightblue', 
                        edgecolor='blue',
                        alpha=0.5,
                        ax=ax)
    
    # Feux
    ax.scatter(summer_fires['longitude'], 
              summer_fires['latitude'],
              c='red', s=10, alpha=0.5,
              label='Feux actifs')
    
    # Athabasca
    ax.scatter(athabasca_info['lon'], 
              athabasca_info['lat'],
              c='green', s=200, marker='*',
              label='Glacier Athabasca')
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Glaciers et feux actifs - Ouest Canadien (Été 2023)')
    ax.legend()
    ax.set_xlim(-130, -110)
    ax.set_ylim(48, 60)
    plt.tight_layout()

## 5. Analyse de proximité

In [ ]:
# Calculer la distance des feux au glacier Athabasca
if 'summer_fires' in locals():
    from geopy.distance import distance
    
    # Calculer les distances
    distances = []
    for idx, fire in summer_fires.iterrows():
        fire_loc = (fire['latitude'], fire['longitude'])
        athabasca_loc = (athabasca_info['lat'], athabasca_info['lon'])
        dist = distance(fire_loc, athabasca_loc).km
        distances.append(dist)
    
    summer_fires['distance_to_athabasca'] = distances
    
    # Feux proches
    nearby_fires = summer_fires[summer_fires['distance_to_athabasca'] < 200]
    print(f"Feux à moins de 200 km d'Athabasca: {len(nearby_fires)}")
    
    # Distribution temporelle
    plt.figure(figsize=(12, 6))
    nearby_fires.groupby(nearby_fires['acq_date'].dt.date).size().plot()
    plt.xlabel('Date')
    plt.ylabel('Nombre de feux détectés')
    plt.title('Feux actifs à moins de 200 km du glacier Athabasca')
    plt.xticks(rotation=45)
    plt.tight_layout()